In [36]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import ast

In [37]:
train_csv = pd.read_csv('D:/UET/ML/mouse_behavior/data/train.csv') 
test_csv = pd.read_csv('D:/UET/ML/mouse_behavior/data/test.csv')
train_annotation_path = 'D:/UET/ML/mouse_behavior/data/train_annotation'
train_tracking_path = 'D:/UET/ML/mouse_behavior/data/train_tracking'
test_tracking_path = 'D:/UET/ML/mouse_behavior/data/test_tracking'

In [38]:
sample_annot_path = 'D:/UET/ML/mouse_behavior/data/train_annotation/AdaptableSnail/44566106.parquet'
sample_annot = pd.read_parquet(sample_annot_path)

sample_annot

,agent_id,target_id,action,start_frame,stop_frame
0,2,2,rear,4,139
1,4,2,avoid,13,52
2,4,4,rear,121,172
3,3,3,rear,156,213
4,4,4,rear,208,261
...,...,...,...,...,...
337,3,1,approach,18172,18184
338,2,2,rear,18195,18287
339,1,4,approach,18266,18305
340,3,3,rear,18297,18363


In [39]:
sample_tracking_path = 'D:/UET/ML/mouse_behavior/data/train_tracking/AdaptableSnail/44566106.parquet'
sample_tracking = pd.read_parquet(sample_tracking_path)

df_frame0 = sample_tracking[sample_tracking["video_frame"] == 5978]
df_frame0

,video_frame,mouse_id,bodypart,x,y
354347,5978,1,body_center,1059.633057,647.893982
354348,5978,1,ear_right,1081.849976,596.245972
354349,5978,1,headpiece_bottombackright,1028.686035,606.973999
354350,5978,1,headpiece_bottomfrontleft,1050.025024,554.635986
354351,5978,1,headpiece_bottomfrontright,1066.619019,579.213013
354352,5978,1,headpiece_topbackleft,1000.265015,576.249023
354353,5978,1,headpiece_topbackright,1017.155029,604.046021
354354,5978,1,headpiece_topfrontleft,1038.597046,548.028015
354355,5978,1,headpiece_topfrontright,1056.302002,578.682007
354356,5978,1,lateral_left,1025.718018,654.656006


In [40]:
train_csv[train_csv['lab_id'].str.startswith('MABe22')]['behaviors_labeled'].count()

0

In [41]:
drop_rows = train_csv[train_csv['lab_id'].str.startswith('MABe22')].index
train_csv_without_MABe = train_csv.drop(drop_rows)
train_csv_without_MABe

,lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,...,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
0,AdaptableSnail,44566106,CD-1 (ICR),white,male,10.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1228,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
1,AdaptableSnail,143861384,CD-1 (ICR),white,male,3.0,8-12 weeks,NaN,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2,AdaptableSnail,209576908,CD-1 (ICR),white,male,7.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1266,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
3,AdaptableSnail,278643799,CD-1 (ICR),white,male,11.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1224,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
4,AdaptableSnail,351967631,CD-1 (ICR),white,male,14.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1204,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8784,UppityFerret,1610683375,C57Bl/6N,black,male,8.0,8 weeks,csds,CD1,white,...,13.0,664,572,38.0,38.0,circular,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""hip_...","[""mouse1,mouse2,reciprocalsniff"", ""mouse1,mous...",DeepLabCut
8785,UppityFerret,1725214092,C57Bl/6N,black,male,27.0,8 weeks,NaN,CD1,white,...,12.8,608,576,38.0,38.0,circular,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""hip_...","[""mouse1,mouse2,intromit"", ""mouse1,mouse2,moun...",DeepLabCut
8786,UppityFerret,1960237444,C57Bl/6N,black,male,14.0,8 weeks,csds,CD1,white,...,12.5,592,576,38.0,38.0,circular,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""hip_...","[""mouse1,mouse2,reciprocalsniff"", ""mouse1,mous...",DeepLabCut
8787,UppityFerret,2045808129,C57Bl/6N,black,male,12.0,8 weeks,csds,CD1,white,...,12.5,624,572,38.0,38.0,circular,neutral,"[""body_center"", ""ear_left"", ""ear_right"", ""hip_...","[""mouse1,mouse2,reciprocalsniff"", ""mouse1,mous...",DeepLabCut


In [42]:
def extract_behavior(train_csv):
    behavior_stats = {
        "total": 0,
        "self-oriented": set(),
        "social-oriented": set(),
    }

    for idx, row in train_csv.iterrows():
        behaviors_str = row['behaviors_labeled']

        if pd.isna(behaviors_str) or behaviors_str.strip() == "":
            continue

        behaviors_list = ast.literal_eval(behaviors_str)

        for behavior in behaviors_list:
            parts = behavior.split(',')
            if len(parts) == 3:
                agent = parts[0].strip()
                target = parts[1].strip()
                action = parts[2].strip()

                if target.lower() == 'self':
                    behavior_stats['self-oriented'].add(action)
                else:
                    behavior_stats['social-oriented'].add(action)

    behavior_stats['total'] = len(behavior_stats['self-oriented']) + len(behavior_stats['social-oriented'])
    return behavior_stats

behavior_stats = extract_behavior(train_csv_without_MABe)
behavior_stats

{'total': 40,
 'self-oriented': {'biteobject',
  'climb',
  'dig',
  'exploreobject',
  'freeze',
  'genitalgroom',
  'huddle',
  'rear',
  'rest',
  'run',
  'selfgroom'},
 'social-oriented': {"'attack'",
  "'dominance'",
  "'sniff'",
  'allogroom',
  'approach',
  'attack',
  'attemptmount',
  'avoid',
  'chase',
  'chaseattack',
  'defend',
  'disengage',
  'dominance',
  'dominancegroom',
  'dominancemount',
  'ejaculate',
  'escape',
  'flinch',
  'follow',
  'intromit',
  'mount',
  'reciprocalsniff',
  'shepherd',
  'sniff',
  'sniffbody',
  'sniffface',
  'sniffgenital',
  'submit',
  'tussle'}}

In [43]:
def extract_bodyparts(train_csv):
    unique_parts = set()

    for idx, row in train_csv.iterrows():
        bodyparts_str = row['body_parts_tracked']

        if pd.isna(bodyparts_str) or bodyparts_str.strip() == "":
            continue

        try:
            bodyparts_list = ast.literal_eval(bodyparts_str)
        except:
            continue

        unique_parts.update(bodyparts_list)

    return len(unique_parts), sorted(unique_parts)

bodyparts_stats = extract_bodyparts(train_csv_without_MABe)
bodyparts_stats

(25,
 ['body_center',
  'ear_left',
  'ear_right',
  'head',
  'headpiece_bottombackleft',
  'headpiece_bottombackright',
  'headpiece_bottomfrontleft',
  'headpiece_bottomfrontright',
  'headpiece_topbackleft',
  'headpiece_topbackright',
  'headpiece_topfrontleft',
  'headpiece_topfrontright',
  'hip_left',
  'hip_right',
  'lateral_left',
  'lateral_right',
  'neck',
  'nose',
  'spine_1',
  'spine_2',
  'tail_base',
  'tail_middle_1',
  'tail_middle_2',
  'tail_midpoint',
  'tail_tip'])